In [1]:
import os

In [2]:
os.chdir(r"D:\Gene_Project")

In [1]:
import pandas as pd

In [4]:
all_files = os.listdir(r"unsupervised_CGC\output")

In [5]:
unsupervised_file_path = r"unsupervised_CGC\output\\"

In [6]:
def read_the_data(file):
    # read one file
    data = pd.read_csv(unsupervised_file_path + file, sep = " ", header = None)
    return data

In [7]:
from joblib import Parallel, delayed

In [8]:
all_files_df = Parallel(n_jobs=6, verbose = 3)(delayed(read_the_data)(i) for i in all_files)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    1.0s
[Parallel(n_jobs=6)]: Done 908 tasks      | elapsed:    2.8s
[Parallel(n_jobs=6)]: Done 4428 tasks      | elapsed:    6.3s
[Parallel(n_jobs=6)]: Done 11596 tasks      | elapsed:   16.4s
[Parallel(n_jobs=6)]: Done 19492 tasks      | elapsed:   28.8s
[Parallel(n_jobs=6)]: Done 19582 out of 19582 | elapsed:   29.0s finished


In [9]:
len(all_files_df)

19582

In [10]:
all_unsupervised = pd.concat(all_files_df, ignore_index = True)

In [11]:
all_unsupervised.columns = ["sequence"]

In [12]:
all_unsupervised.head()

,sequence
0,"1.A.72,MerR,GH23"
1,"3.A.1,CE4"
2,"3.A.23,9.A.5,MCPsignal,2.A.21,2.A.22,TetR_N,3...."
3,"CE4,8.A.5,3.A.1,3.A.1,3.A.1"
4,"GT51,Peripla_BP_2,3.A.1,3.A.1,9.B.169"


In [31]:
all_unsupervised.to_csv("all_unsupervised_genes.csv", index = False)

In [13]:
all_unsupervised_split = [seq.split(",") for seq in all_unsupervised["sequence"]]

In [14]:
all_genes = [gene for seq_split in all_unsupervised_split for gene in seq_split]

In [15]:
from collections import Counter

In [16]:
unsupervised_genes = Counter(all_genes)

In [17]:
len(Counter(all_genes))

6927

In [18]:
sig_gene = pd.read_csv(r"signature_gene_reference.tsv", sep = "\t", header = None)

In [19]:
sig_gene[sig_gene[1] == "Nitroreductase"]

,0,1
1141,STP,Nitroreductase


In [41]:
# read the new data

In [20]:
latest_seq_data = pd.read_csv(r"pul_seq_low_high_substr_year.tsv", sep = "\t")

In [21]:
latest_seq_data.head()

,PULid,sig_gene_seq,low_level_substr,high_level_substr,Pub_year
0,PUL0001,"GH1,8.A.49,CE2,GH130,GH130,3.A.1,3.A.1,SBP_bac...",beta-mannan,beta-mannan,2019
1,PUL0002,GH16,lichenan,beta-glucan,1996
2,PUL0003,"GH30_8,GH43_16|CBM6",xylan,xylan,2016
3,PUL0004,"4.A.1,GH1","glucose,cellobiose,maltose",multiple_substrates,2016
4,PUL0005,"GH94,GH3","beta-glucan,sophorose,laminaribiose",multiple_substrates,2016


In [53]:
latest_seq_data["low_level_substr"].value_counts()

capsule polysaccharide                  60
O-antigen                               37
pectin                                  28
xylan                                   25
alginate                                21
                                        ..
lichenan                                 1
exopolysaccharide,lipopolysaccharide     1
melezitose                               1
maltooligosaccharide                     1
glucuronoarabinoxylan                    1
Name: low_level_substr, Length: 170, dtype: int64

In [22]:
latest_seq_data["high_level_substr"].value_counts()

multiple_substrates           139
mono/di/trisaccharide          78
capsule polysaccharide         60
algal glycans                  40
pectin                         38
xylan                          38
O-antigen                      37
galactan                       36
alpha-glucan                   23
beta-mannan                    19
cellulose                      18
chitin                         15
glycosaminoglycan              14
beta-glucan                    13
N-glycan                       12
exopolysaccharide              10
-                               9
fructan                         8
alpha-mannan                    8
host glycan                     7
xyloglucan                      7
plant polysaccharide            6
glycoprotein                    6
human milk oligosaccharide      5
hemicellulose                   5
O-glycan                        3
Name: high_level_substr, dtype: int64

In [23]:
# all genes available
split_genes_supervised = [seq.split(",") for seq in latest_seq_data["sig_gene_seq"].astype("str")]

In [24]:
# all genes
all_genes_supervised = [gene for seq in split_genes_supervised for gene in seq]

In [25]:
# count
supervised_genes = Counter(all_genes_supervised)

In [26]:
len(supervised_genes)

526

In [1]:
# unsupervised_genes

In [28]:
supervised_in_unsupervised = dict((k, v) for k, v in unsupervised_genes.items() if k in supervised_genes.keys())

In [2]:
# supervised_in_unsupervised

In [30]:
len(supervised_in_unsupervised)

500

In [31]:
missing_genes = dict((k, v) for k, v in supervised_genes.items() if k not in unsupervised_genes.keys())

In [32]:
missing_genes

{'CE10': 1,
 'HTH_8|HTH_AraC': 1,
 'nan': 1,
 'GH18|CBM14': 1,
 'CE10|CE8': 2,
 'Response_reg|Reg_prop|HTH_18|HisKA|Y_Y_Y|HATPase_c': 1,
 'CBM54|GH16|CBM4|CBM4|CBM6|CBM4|CBM4': 1,
 'GH43_18|GH43_34|CBM32': 1,
 'PL5|PL7': 2,
 'GH146|GH97': 1,
 'GH51|GH43_29': 1,
 'GH26|GH5_4': 1,
 'Mga|HTH_Mga|PRD_Mga': 2,
 'GH16|CBM16': 1,
 'GH5_8|CBM3|CBM3|CBM3|GH5_1': 2,
 'CBM22|CBM22|GH10|CBM3|CBM3|GH5_1': 1,
 'GH10|CBM3|GH12|GH48': 1,
 'CE6|GH151': 1,
 'GH16|CBM16|CBM16': 1,
 'HTH_AraC|HTH_11': 1,
 'GT46': 2,
 'GH43_24|GH16': 1,
 'GH16|GH43_24': 1,
 'CBM82|CBM83|GH13_41': 1,
 'CBM4|CBM4|GH10': 1,
 '-': 2}

In [33]:
from gensim.test.utils import datapath
from gensim import utils


In [82]:
# class MyCorpus:
#     """An iterator that yields sentences (lists of str)."""

#     def __iter__(self):
#         corpus_path = all_unsupervised["sequence"].values
#         for line in corpus_path:
#             # assume there's one document per line, tokens separated by whitespace
#             yield utils.simple_preprocess(line)

In [34]:
import gensim.models

# sentences = MyCorpus()

In [35]:
from gensim.models import Phrases

In [36]:
bigram_transformer = Phrases(all_unsupervised_split)

In [37]:
len(bigram_transformer.export_phrases())

762

In [38]:
model = gensim.models.Word2Vec(sentences=bigram_transformer[all_unsupervised_split], epochs = 6, workers = 6, vector_size = 100)

In [39]:
model.wv.most_similar(positive = [ 'LacI|Peripla_BP_3'])

[('Peripla_BP_3|LacI', 0.9932994842529297),
 ('LacI|Peripla_BP_1', 0.5442603826522827),
 ('Peripla_BP_1|LacI', 0.5279130935668945),
 ('LacI|Peripla_BP_3|Peripla_BP_1', 0.5088406205177307),
 ('Peripla_BP_1|LacI|Peripla_BP_3', 0.4964331388473511),
 ('Peripla_BP_3|Peripla_BP_1|LacI', 0.4933910369873047),
 ('Peripla_BP_1|Peripla_BP_3|LacI', 0.4923916161060333),
 ('Peripla_BP_3|LacI|Peripla_BP_1', 0.4862915277481079),
 ('GH85|CBM32', 0.45317068696022034),
 ('LacI|Peripla_BP_1|Peripla_BP_3', 0.4333457946777344)]

In [40]:
len(model.wv.key_to_index)

4055

In [41]:
split_genes_supervised_phrases = bigram_transformer[split_genes_supervised]

In [42]:
catch_supervised_with_ngrams = []

for i in range(0, len(split_genes_supervised)): 
    catch_supervised_with_ngrams.append(split_genes_supervised_phrases[i])

In [43]:
tokenized_supervised_seqs = catch_supervised_with_ngrams

In [44]:
len(tokenized_supervised_seqs)

654

In [45]:
supervised_unique_genes = [gene for split_genes in tokenized_supervised_seqs for gene in split_genes]

In [46]:
len(supervised_unique_genes)

4253

In [47]:
word2vec_index = model.wv.index_to_key

In [3]:
# word2vec_index

In [4]:
# model.wv.key_to_index

In [50]:
model.wv.vectors[0]

array([ 0.12659074, -0.09510124,  0.15970372, -0.8029479 , -0.6253466 ,
       -0.46601698,  0.44897443,  0.3036746 ,  1.2778811 , -0.96759725,
       -0.2380571 , -0.44331366,  1.5555876 , -0.4288723 ,  0.5845584 ,
        0.91753423, -0.18391268, -0.5322133 ,  0.48250985,  0.3295316 ,
       -0.3022187 ,  0.7132207 ,  1.260019  , -1.3358107 , -1.3056176 ,
       -0.16846363,  0.17787117,  1.0617414 , -0.77237   ,  0.65428644,
       -0.7843513 , -0.44390485,  0.9641004 ,  0.35507268, -0.46157923,
        0.18406269, -1.3676366 , -0.9302938 , -0.74472725,  0.07356495,
       -0.8849165 , -0.22421247,  0.90953726, -1.435987  , -1.3562899 ,
        1.6576016 ,  1.0095739 , -0.7544265 , -0.5913958 , -1.8287057 ,
        1.4473157 ,  1.6175896 , -0.78792614,  0.4287036 , -0.40822038,
       -0.04388471, -1.3559475 ,  0.0662647 , -0.26487038,  1.2185138 ,
       -0.7122932 , -0.16895469, -0.28428933, -1.526949  ,  0.8039583 ,
       -0.08726908, -1.3048252 , -0.97025895,  0.81603897, -0.81

In [51]:
word_2_vec_dict = {}
for word in supervised_unique_genes: 
    if word in word2vec_index: 
        index = model.wv.key_to_index[word]
        word_vec = model.wv.vectors[index]
        word_2_vec_dict[word] = word_vec

In [1]:
# word_2_vec_dict